In [5]:
import pandas as pd  
  
# 读取Excel文件  
file_path = '专题统计_实际控制人持股变动.xlsx'  # 请替换为你的Excel文件的路径  
df = pd.read_excel(file_path, engine='openpyxl', dtype={'证券代码': str})
# 筛选实际控制人为国务院国有资产监督管理委员会的股票   
# 创建一个包含要搜索的所有关键字的列表  
keywords = ['国有', '国务院', '人民政府', '中共']  
# 使用str.contains()函数检查“实际控制人名称”列中的值是否包含关键字列表中的任何一个关键字，并使用any(axis=1)将结果合并为一行  
filtered_df = df[df['实际控制人名称'].str.contains('|'.join(keywords), na=False)]    
    
# 提取证券代码列并转换为列表    
stock_codes = filtered_df['证券代码'].astype(str).tolist()  
# 提取证券代码列并转换为列表  
stock_codes = filtered_df['证券代码'].astype(str).tolist()  
  
# 使用列表推导式筛选出以"00"开头的股票代码  
stock_codes_00_prefix = [code for code in stock_codes if code.startswith("00")]  
  
print(stock_codes_00_prefix)

['000966', '000031', '002423', '000028', '000959', '002053', '000792', '000906', '002232', '000581', '000969', '000507', '002066', '002349', '000032', '000050', '000059', '000400', '000404', '000423', '000521', '000537', '000543', '000026', '000635', '000151', '000733', '000736', '000519', '000778', '000815', '000859', '000987', '000701', '000725', '000727', '002039', '000738', '000777', '002110', '000930', '000999', '002200', '002204', '002051', '002109', '002128', '002189', '000768', '000960', '000591', '000958', '000720', '000875', '000401', '000717', '000856', '002097', '002037', '000762', '000410', '000825', '000755', '000937', '000503', '000529', '002163', '000737', '000983', '000016', '000025', '000029', '000045', '000058', '000061', '000069', '000089', '000096', '000417', '000421', '000422', '000425', '000428', '000488', '000501', '000505', '000514', '000523', '000528', '000532', '000561', '000570', '000599', '000600', '000630', '000652', '000682', '000698', '000705', '000709',

In [6]:
stock_codes = []
# 股票代码列表    
for i in stock_codes_00_prefix:
    stock_codes.append('sz.'+i)
stock_codes

['sz.000966',
 'sz.000031',
 'sz.002423',
 'sz.000028',
 'sz.000959',
 'sz.002053',
 'sz.000792',
 'sz.000906',
 'sz.002232',
 'sz.000581',
 'sz.000969',
 'sz.000507',
 'sz.002066',
 'sz.002349',
 'sz.000032',
 'sz.000050',
 'sz.000059',
 'sz.000400',
 'sz.000404',
 'sz.000423',
 'sz.000521',
 'sz.000537',
 'sz.000543',
 'sz.000026',
 'sz.000635',
 'sz.000151',
 'sz.000733',
 'sz.000736',
 'sz.000519',
 'sz.000778',
 'sz.000815',
 'sz.000859',
 'sz.000987',
 'sz.000701',
 'sz.000725',
 'sz.000727',
 'sz.002039',
 'sz.000738',
 'sz.000777',
 'sz.002110',
 'sz.000930',
 'sz.000999',
 'sz.002200',
 'sz.002204',
 'sz.002051',
 'sz.002109',
 'sz.002128',
 'sz.002189',
 'sz.000768',
 'sz.000960',
 'sz.000591',
 'sz.000958',
 'sz.000720',
 'sz.000875',
 'sz.000401',
 'sz.000717',
 'sz.000856',
 'sz.002097',
 'sz.002037',
 'sz.000762',
 'sz.000410',
 'sz.000825',
 'sz.000755',
 'sz.000937',
 'sz.000503',
 'sz.000529',
 'sz.002163',
 'sz.000737',
 'sz.000983',
 'sz.000016',
 'sz.000025',
 'sz.0

In [8]:
import baostock as bs    
import pandas as pd    
import os  
  
# 登陆系统    
lg = bs.login()    
# 显示登陆返回信息    
print('login respond error_code:' + lg.error_code)    
print('login respond  error_msg:' + lg.error_msg)    
    

  
# 指定存放Excel文件的文件夹路径  
folder_path = "国企深市的季频偿债能力"  # 替换为你的文件夹路径  
  
# 如果文件夹不存在，创建它  
if not os.path.exists(folder_path):  
    os.makedirs(folder_path)  
  
# 遍历每个股票    
for code in stock_codes:    
    print(f"Processing {code}")  
    all_data_list = []    
    # 遍历每一年的四个季度    
    for year in range(2007, 2024):  # 2024用于包括2023年的数据    
        for quarter in range(1, 5):  # 一年有四个季度    
            print(f"Processing {year} quarter {quarter}")    
            # 季频偿债能力  
            balance_list = []    
            rs_balance_flow = bs.query_balance_data(code=code, year=year, quarter=quarter)    
            while (rs_balance_flow.error_code == '0') & rs_balance_flow.next():    
                balance_list.append(rs_balance_flow.get_row_data())    
            # 将数据添加到列表中    
            result_cash_flow = pd.DataFrame(balance_list, columns=rs_balance_flow.fields)    
            result_cash_flow['code'] = code    
            result_cash_flow['year'] = year    
            result_cash_flow['quarter'] = quarter    
            all_data_list.append(result_cash_flow)    
              
    # 合并该股票的所有数据    
    all_data = pd.concat(all_data_list, ignore_index=True)    
    # 将该股票的数据输出到单独的Excel文件中  
    excel_file_path = os.path.join(folder_path, f"{code}.xlsx") # 创建该股票的Excel文件路径  
    all_data.to_excel(excel_file_path, index=False) # 将数据保存到该Excel文件中  
    print(f"Data for {code} saved to {excel_file_path}\n") # 输出保存信息  
        
# 登出系统    
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
Processing sz.000966
Processing 2007 quarter 1
Processing 2007 quarter 2
Processing 2007 quarter 3
Processing 2007 quarter 4
Processing 2008 quarter 1
Processing 2008 quarter 2
Processing 2008 quarter 3
Processing 2008 quarter 4
Processing 2009 quarter 1
Processing 2009 quarter 2
Processing 2009 quarter 3
Processing 2009 quarter 4
Processing 2010 quarter 1
Processing 2010 quarter 2
Processing 2010 quarter 3
Processing 2010 quarter 4
Processing 2011 quarter 1
Processing 2011 quarter 2
Processing 2011 quarter 3
Processing 2011 quarter 4
Processing 2012 quarter 1
Processing 2012 quarter 2
Processing 2012 quarter 3
Processing 2012 quarter 4
Processing 2013 quarter 1
Processing 2013 quarter 2
Processing 2013 quarter 3
Processing 2013 quarter 4
Processing 2014 quarter 1
Processing 2014 quarter 2
Processing 2014 quarter 3
Processing 2014 quarter 4
Processing 2015 quarter 1
Processing 2015 quarter 2
Processing 2015 quar